# 二 ⼿动⽣成⼆分类任务的数据集

## 0 数据生成

In [1]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

devc = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 参数设置
n_class = 2  # 类别数
n_train = 7000  # 训练集样本数
n_test = 3000  # 测试集样本数
n_total = n_train + n_test  # 总样本数
p = 200  # 特征维度
intercept = 0.028  # 截距项
coef = 0.0056  # 每个特征的系数
noise_mean = 2
noise_std = 1.0  # 噪声标准差

n_data = torch.ones(n_total, p, device=devc)
x1 = torch.normal(mean=noise_mean * n_data, std=noise_std)
x2 = torch.normal(mean=-noise_mean * n_data, std=noise_std)
X = torch.cat((x1, x2), dim=0)
y1 = torch.ones(n_total, dtype=torch.long, device=devc)
y2 = torch.zeros(n_total, dtype=torch.long, device=devc)
y = torch.cat((y1, y2), dim=0)

X_train = X.index_select(0, [0, n_train])
y_train = y.index_select(0, [0, n_train])
X_test = X.index_select(0, [n_train, n_test])
y_test = y.index_select(0, [n_train, n_test])


def train_iter(batch_size, features=X_train, labels=y_train):
    num_examples = len(features)
    indicies = torch.randperm(num_examples)
    for i in range(0, num_examples, batch_size):
        batch_indices = indicies[i : min(i + batch_size, num_examples)]
        yield features[batch_indices], labels[batch_indices]


def test_iter(batch_size, features=X_test, labels=y_test):
    num_examples = len(features)
    indicies = torch.arange(0, num_examples)
    for i in range(0, num_examples, batch_size):
        batch_indices = indicies[i : min(i + batch_size, num_examples)]
        yield features[batch_indices], labels[batch_indices]